# 번역 함수

In [1]:
# 구글키.json 경로 설정하기
import json 

with open("./data/GoogleKey.json") as fr:
    google_secret = json.loads(fr.read())

In [2]:
# Google Translation API 사용
from google.oauth2.service_account import Credentials
from google.cloud import translate

credentials = Credentials.from_service_account_info(google_secret)
google_translate_client = translate.TranslationServiceClient(credentials=credentials)

In [3]:
# Google 번역 결과 가져오는 함수
def get_translation(query):
    parent = f"projects/{google_secret['project_id']}/locations/global"
    response = google_translate_client.translate_text(
        request={
            "parent": parent,
            "contents": [query],
            "mime_type": "text/plain",
            "source_language_code": "ko",
            "target_language_code": "en-US",
        }
    )
    return response.translations[0].translated_text

In [4]:
import csv
import pandas as pd

In [17]:
# 브랜드 별 파일 이름 리스트 
file_names = [
    "[함께하닭] 헬스 스팀 소스 닭가슴살",
    "함께하닭 닭가슴살 소시지 청양고추 100g",
    "함께하닭 헬스 스팀 닭가슴살 카레맛 100g",
    "함께하닭 헬스앤 돈다리살 100g"
]

In [18]:
# 리뷰를 번역하여 데이터프레임에 할당
def translate_review(file):
    file_path = f"./total_preprocessed_review/HamkkeHadak/{file}" + ".csv"
    df = pd.read_csv(file_path, encoding="utf-8")
    for i in range(len(df)):
        review = df.review.iloc[i]
        translated_review = get_translation(review)
        df.loc[i, "translated_review"] = translated_review
    print("translate review 끝")
    return df

In [19]:
# 파일 저장 함수
def save_file(dataframe, file):
    translated_df = dataframe
    file_path = f"./total_preprocessed_review/HamkkeHadak/{file}" + "_translated.csv"
    translated_df.to_csv(file_path, encoding="utf-8")
    print("file_path", file_path)

In [ ]:
# 최종적으로 사용하는 번역 함수

from tqdm import tqdm

def translate_start():
    # tqdm을 사용하여 enumerate를 감싸서 진행 상황을 모니터링
    for i, file in enumerate(tqdm(file_names, desc="Translating files")):
        translated_df = translate_review(file)
        save_file(translated_df, file)
    print("translate start 끝")

translate_start()

Translating files:  25%|████████████████▎                                                | 1/4 [00:15<00:46, 15.37s/it]

translate review 끝
file_path ./total_preprocessed_review/HamkkeHadak/[함께하닭] 헬스 스팀 소스 닭가슴살_translated.csv


Translating files:  50%|████████████████████████████████▌                                | 2/4 [00:20<00:18,  9.10s/it]

translate review 끝
file_path ./total_preprocessed_review/HamkkeHadak/함께하닭 닭가슴살 소시지 청양고추 100g_translated.csv
